In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_pinecone import PineconeVectorStore, PineconeEmbeddings
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os

D:\4Kurs\1Sem\NN\Lab3\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
api_key_pinecone = os.getenv("PINECONE_API_KEY")

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=api_key
)

In [4]:
pc = Pinecone(api_key=api_key_pinecone)

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)

index_name = "nn-lab-3"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=spec
    )

# See that it is empty
print("Index before upsert:")
print(pc.Index(index_name).describe_index_stats())
print("\n")

Index before upsert:
{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}




In [5]:
docs = []
for path in ["./data/file1.pdf"]:
    loader = PyPDFLoader(path)
    docs.extend(loader.load())

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = splitter.split_documents(docs)

In [6]:
namespace = "text"

docsearch = PineconeVectorStore.from_documents(
    documents=texts,
    index_name=index_name,
    embedding=embeddings,
    namespace=namespace
)

print("Index after upsert:")
print(pc.Index(index_name).describe_index_stats())
print("\n")

Index after upsert:
{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'text': {'vector_count': 6}},
 'total_vector_count': 6,
 'vector_type': 'dense'}




In [8]:
index = pc.Index(index_name)

for ids in index.list(namespace=namespace):
    query = index.query(
        id=ids[0],
        namespace=namespace,
        top_k=1,
        include_values=True,
        include_metadata=True
    )
    print(query)
    print("\n")

{'matches': [{'id': '40071fee-4e58-4298-8351-70d351a177c9',
              'metadata': {'author': 'Соломія Дребот',
                           'creationdate': '2025-09-29T03:20:08+03:00',
                           'creator': 'Microsoft® Word 2016',
                           'moddate': '2025-09-29T03:20:08+03:00',
                           'page': 0.0,
                           'page_label': '1',
                           'producer': 'Microsoft® Word 2016',
                           'source': 'file1.pdf',
                           'text': 'повертається. \n'
                                   '2.5. Ціни на проживання можуть змінюватись '
                                   'залежно від сезону та \n'
                                   'завантаженості готелю. \n'
                                   '3. Правила поведінки в готелі \n'
                                   '3.1. В готелі заборонено: \n'
                                   '• Палити в номерах та громадських '
                 

In [11]:
retriever=docsearch.as_retriever(search_kwargs={"k": 3})

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=api_key,
    temperature=0.0
)

prompt = ChatPromptTemplate.from_template(
    "Використай наведений контекст, щоб відповісти стисло і по суті.\n"
    "Контекст:\n{context}\n\nПитання: {question}"
)

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [12]:
question = "Поясни головну ідею того що пояснюється в файлі."
answer = rag_chain.invoke(question)

print("Q:", question)
print("A:", answer)

Q: Поясни головну ідею того що пояснюється в файлі.
A: Головна ідея файлу полягає в встановленні правил проживання в бутик-готелі «Rarity», які регулюють поведінку гостей, відповідальність за майно, умови бронювання та оплати, а також графік роботи послуг готелю. Ці правила спрямовані на забезпечення комфортного та безпечного перебування для всіх гостей.
